In [1]:
import pandas as pd
# import matplotlib.pyplot as plt
# from scipy.interpolate import interp1d
import numpy as np
import pandas as pd
import os
import dask.dataframe as ddf
import re

import sys
sys.path.append("..")

from channeling_lib import AWS_file_loader, load_path, filter_data_based_on_time

stations_str = ['Tom Joad','Rosanna','Bette Davis', 'Layla', 'Mrs Robinson']

# TinyTag_str = ['CEB_1', 'CEB_2', 'CEB_3', 'CEB_4', 'CEB_5', 'TH1', 'TH2', 'TH3', 'TH5', 'TH6', 'TH8', 'TT1', 'TT2', 'TT3', 'TT4', 'TT5', 'TT6', 'TT7', 'TT9', 'TT12', 'TT13', 'TT14', 'TT15', 'TT16', 'TT17', 'TT18']

In [2]:
path = load_path()+'RawData/TinyTag/'

instrument_textbook_name = load_path()+'instrument_textbook_BLcourse_spring2025(BL instruments).csv'
instrument_textbook_data = pd.read_csv(instrument_textbook_name, encoding='latin1')

# instrument_textbook_data['Station name']

# Formatting instrument textbook

This section of the code ensures consistency no matter what chnages in the excel file in sharepoint

In [3]:
# Extract station names (keep only prefix + number)
instrument_textbook_data['Formatted Name'] = [
    re.sub(r"(TT|TH|CEB)(\d+).*", r"\1\2", name) if re.match(r"(TT|TH|CEB)\d+", name) else name
    for name in instrument_textbook_data['Station name']
]

# Find duplicate names
duplicates = instrument_textbook_data['Formatted Name'].value_counts()
duplicate_names = duplicates[duplicates > 1].index  # Names that appear more than once

# Apply "_low" and "_high" based on "Th height (m)"
for name in duplicate_names:
    subset = instrument_textbook_data[instrument_textbook_data['Formatted Name'] == name]
    
    # Find the lowest and highest Th height
    min_index = subset['Th height (m)'].idxmin()
    max_index = subset['Th height (m)'].idxmax()
    
    # Rename them
    instrument_textbook_data.loc[min_index, 'Formatted Name'] = f"{name}_low"
    instrument_textbook_data.loc[max_index, 'Formatted Name'] = f"{name}_high"

# Get final formatted list
formatted_names = instrument_textbook_data['Formatted Name'].tolist()

# formatted_names

The following cell can ONLY be run once. If the following line soudl be 're-run', restart the kernal and run all

In [4]:
# Insert 'Formatted Name' as the first column
instrument_textbook_data.insert(0, 'Formatted Station Name', instrument_textbook_data.pop('Formatted Name'))

# Loading start and end times of Tinytag for calibration

In [5]:
# TinyTag_str = instrument_textbook_data['Station name'].tolist()
TinyTag_str = [s for s in formatted_names if s.startswith(("TT", "TH", "CEB"))]


# TinyTag_str

In [8]:
manual_times = {}

for station in TinyTag_str:
    setup_time_idx = pd.to_datetime(instrument_textbook_data.loc[instrument_textbook_data['Formatted Station Name'] == station, 'Setup time (UTC)'].values[0])

    maintenance_start_time_idx = pd.to_datetime(instrument_textbook_data.loc[instrument_textbook_data['Formatted Station Name'] == station, 'Maintenance start time (UTC)'].values[0])

    # maintenance_duration_idx = int(instrument_textbook_data.loc[instrument_textbook_data['Formatted Station Name'] == station, 'Maintenance duration (minutes)'].values[0])
    maintenance_duration_value = instrument_textbook_data.loc[instrument_textbook_data['Formatted Station Name'] == station, 'Maintenance duration (minutes)'].values[0]
    maintenance_duration_idx = int(maintenance_duration_value) if not pd.isna(maintenance_duration_value) else 0

    retrieval_time_idx = pd.to_datetime(instrument_textbook_data.loc[instrument_textbook_data['Formatted Station Name'] == station, 'Retrieval time (UTC)'].values[0])

    manual_times[station] = {
            'setup_time': setup_time_idx,
            'maintenance_start_time': maintenance_start_time_idx,
            'maintenance_duration': maintenance_duration_idx,
            'retrieval_time': retrieval_time_idx
        }

# manual_times

# Loading TinyTag  data

The TinyTag data is loaded using UNISasci function read_Tinytag(). The data is then loaded dynamically using load_TinyTag_folders() function (Alberte)

- Calibration data
    - ceb_calibration_data = load_TinyTag_folders(path, 'CEB', 'calibration')
    - TH_calibration_data = load_TinyTag_folders(path, 'TH', 'calibration')
    - TT_calibration_data = load_TinyTag_folders(path, 'TT', 'calibration')

- Actual data
    - ceb_data = load_TinyTag_folders(path, 'CEB', 'normal')
    - TH_data = load_TinyTag_folders(path, 'TH', 'normal')
    - TT_data = load_TinyTag_folders(path, 'TT', 'normal')


Specific data is selected by:
- tag_data['tag_i_normal_data']
- tag_calibration_data['tag_i_calibration_data']

Where i corresponds to the TinyTag number in teh raw data folder



In [9]:
#function from unis github
def read_Tinytag(filename, sensor):
    '''
    Reads data from one or several data files from the Tinytag output files.

    Parameters:
    -------
    filename: str
        String with path to file(s)
        If several files shall be read, specify a string including UNIX-style wildcards
    sensor: str
        One of "TT", "TH" or "CEB"
    Returns
    -------
    df : pandas dataframe
        a pandas dataframe with time as index and the individual variables as columns.
    '''

    import dask.dataframe as ddf


    if sensor == "TT":
        df = ddf.read_csv(filename, delimiter="\t", skiprows=5, parse_dates=[1], date_format="%d %b %Y %H:%M:%S", names=["RECORD", "TIMESTAMP", "T_black", "T_white"], encoding = "ISO-8859-1")
    elif sensor == "TH":
        df = ddf.read_csv(filename, delimiter="\t", skiprows=5, parse_dates=[1], date_format="%d %b %Y %H:%M:%S", names=["RECORD", "TIMESTAMP", "T", "RH"], encoding = "ISO-8859-1")
    elif sensor == "CEB":
        df = ddf.read_csv(filename, delimiter="\t", skiprows=5, parse_dates=[1], date_format="%d %b %Y %H:%M:%S", names=["RECORD", "TIMESTAMP", "T"], encoding = "ISO-8859-1")
    else:
        assert False, 'Sensortype of Tinytag not known. Should be one of "TT", "TH" or "CEB".'

    df = df.compute()
    df.set_index("TIMESTAMP", inplace=True)

    for key in list(df.columns):
        if key == "RECORD":
            pass
        else:
            data = [float(i.split(" ")[0]) for i in df[key]]
            unit = df[key].iloc[0].split(" ")[1]
            if unit == "°C":
                unit = "degC"
            new_key = f"{key}_{unit}"

            df[new_key] = data

            df.drop(key, axis=1, inplace=True)

    return df

In [10]:
# # Get all CEB_i folders inside TinyTag
# folders = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f)) and f.startswith('CEB_')]

# # Dictionary to store the datasets
# calibration_data = {}

# # Loop through each CEB_i folder
# for folder in folders:
#     folder_path = os.path.join(path, folder)
    
#     # Get all .txt files that start with "CEB_i_calibration_"
#     files = [f for f in os.listdir(folder_path) if f.startswith(folder + "_calibration_") and f.endswith(".txt")]
    
#     for file in files:
#         file_path = os.path.join(folder_path, file)
        
#         # Run the read_Tinytag function
#         dataset = read_Tinytag(file_path, 'CEB')
        
#         # Store the dataset in a dictionary using the folder name as the key
#         calibration_data[f"{folder}_calibration_data"] = dataset

# # Loop over the stored datasets and assign them as individual variables
# for dataset_name in calibration_data.keys():
#     globals()[dataset_name] = calibration_data[dataset_name]

In [11]:
# # TH
# folders = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f)) and f.startswith('TH')]

# # Dictionary to store the datasets
# calibration_data = {}

# for folder in folders:
#     folder_path = os.path.join(path, folder)
    
#     # Get all .txt files that start with "THi_calibration_"
#     files = [f for f in os.listdir(folder_path) if f.startswith(folder + "_calibration_") and f.endswith(".txt")]
    
#     for file in files:
#         file_path = os.path.join(folder_path, file)
        
#         # Run the read_Tinytag function
#         dataset = read_Tinytag(file_path, 'TH')
        
#         # Store the dataset in a dictionary using the folder name as the key
#         calibration_data[f"{folder}_calibration_data"] = dataset

# for dataset_name in calibration_data.keys():
#     globals()[dataset_name] = calibration_data[dataset_name]

In [12]:
# # TT
# folders = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f)) and f.startswith('TT')]

# # Dictionary to store the datasets
# calibration_data = {}

# for folder in folders:
#     folder_path = os.path.join(path, folder)
    
#     # Get all .txt files that start with "TTi_calibration_"
#     files = [f for f in os.listdir(folder_path) if f.startswith(folder + "_calibration_") and f.endswith(".txt")]
    
#     for file in files:
#         file_path = os.path.join(folder_path, file)
        
#         # Run the read_Tinytag function
#         dataset = read_Tinytag(file_path, 'TT')
        
#         # Store the dataset in a dictionary using the folder name as the key
#         calibration_data[f"{folder}_calibration_data"] = dataset

# for dataset_name in calibration_data.keys():
#     globals()[dataset_name] = calibration_data[dataset_name]

In [13]:
def load_TinyTag_folders(path, tag, datatype):
    """
    Processes folders that start with a specific prefix, reads corresponding files,
    and stores their datasets in a dictionary.
    """
    # prefix = 0
    # prefix = []
    if tag == 'CEB':
        prefix = 'CEB_'
    elif tag == 'TH':
        prefix = 'TH'
    elif tag == 'TT':
        prefix = 'TT'

    if datatype == 'calibration':
        folders = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f)) and f.startswith(prefix)]
        calibration_data = {}

        for folder in folders:
            folder_path = os.path.join(path, folder)
            files = [f for f in os.listdir(folder_path) if f.startswith(f"{folder}_calibration_") and f.endswith(".txt")]
            
            for file in files:
                file_path = os.path.join(folder_path, file)
                dataset = read_Tinytag(file_path, tag)  # Process the file
                calibration_data[f"{folder}_calibration_data"] = dataset
        
        return calibration_data
    
    elif datatype == 'normal':
        folders = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f)) and f.startswith(prefix)]
        normal_data = {}

        for folder in folders:
            folder_path = os.path.join(path, folder)
            # Filter files that DO NOT contain "_calibration" in their name
            files = [f for f in os.listdir(folder_path) if "_calibration" not in f and f.endswith(".txt")]
            
            for file in files:
                file_path = os.path.join(folder_path, file)
                dataset = read_Tinytag(file_path, tag)  # Process the file
                normal_data[f"{folder}_normal_data"] = dataset  # Different key to distinguish

        return normal_data

        
# # Process all folder types
# all_data = {}
# for prefix, tag in zip(["CEB_", "TH", "TT"], ["CEB", "TH", "TT"]):
#     all_data.update(load_TinyTag_folders(path, prefix, tag))

# # Assign the datasets to global variables
# globals().update(all_data)


In [17]:
ceb_calibration_data = load_TinyTag_folders(path, 'CEB', 'calibration')
TH_calibration_data = load_TinyTag_folders(path, 'TH', 'calibration')
TT_calibration_data = load_TinyTag_folders(path, 'TT', 'calibration')

ceb_data = load_TinyTag_folders(path, 'CEB', 'normal')
TH_data = load_TinyTag_folders(path, 'TH', 'normal')
TT_data = load_TinyTag_folders(path, 'TT', 'normal')

# ceb_calibration_data['CEB_1_calibration_data']

ceb_data['CEB_1_normal_data']



,RECORD,T_degC
TIMESTAMP,,
2025-01-26 15:18:44,1,15.29
2025-01-26 15:19:44,2,15.93
2025-01-26 15:20:44,3,15.84
2025-01-26 15:21:44,4,15.80
2025-01-26 15:22:44,5,15.74
...,...,...
2025-02-03 09:22:44,11165,0.05
2025-02-03 09:23:44,11166,0.16
2025-02-03 09:24:44,11167,0.17
